In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
import numpy as np
import os
import gc   
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array



combined_df = pd.read_csv('/root/ubc_ocean/alper/models/similar_tmas/similar_images_final_combined.csv')


class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, image_paths, batch_size=32, dim=(224, 224), n_channels=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.image_paths = image_paths
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        image_paths_temp = [self.image_paths[k] for k in indexes]

        # Generate data
        X = self.__data_generation(image_paths_temp)

        return X, X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_paths_temp):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, img_path in enumerate(image_paths_temp):
            # Store sample
            img = load_img(img_path, target_size=self.dim)
            X[i,] = img_to_array(img) / 255.0

        return X

# Parameters
params = {'dim': (224, 224),
          'batch_size': 32,
          'n_channels': 3,
          'shuffle': True}

# Datasets
image_paths = combined_df['Similar_Image_Path'].tolist()  # Assuming this is your dataset

# Generators
training_generator = DataGenerator(image_paths, **params)


gc.collect()
gc.set_threshold(10000)

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D
from keras.models import Model

def build_simple_autoencoder(input_shape=(224, 224, 3)):
    # Encoder
    input_img = Input(shape=input_shape)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    encoded = Dense(64, activation='relu')(x)

    # Decoder
    x = Dense(56 * 56 * 8, activation='relu')(encoded)
    x = Reshape((56, 56, 8))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return autoencoder

# Build and compile the simpler autoencoder
simple_autoencoder = build_simple_autoencoder()
simple_autoencoder.summary()



In [ ]:
simple_autoencoder.fit(training_generator, epochs=10, validation_data=training_generator)
simple_autoencoder.save('CheckAutoencoderSimple.h5')


In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D, BatchNormalization
from keras.models import Model

def build_enhanced_autoencoder(input_shape=(224, 224, 3)):
    # Encoder
    input_img = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    encoded = Dense(128, activation='relu')(x)
    # Decoder
    x = Dense(14 * 14 * 128, activation='relu')(encoded)  # Измените размер, если необходимо
    x = Reshape((14, 14, 128))(x)
    x = UpSampling2D((2, 2))(x)  # Увеличиваем до 28x28
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)  # Увеличиваем до 224x224, если это последний шаг увеличения
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return autoencoder

# Build and compile the autoencoder
autoencoder = build_enhanced_autoencoder()
autoencoder.summary()


In [8]:
#autoencoder.fit(training_generator, epochs=10, validation_data=training_generator)
#autoencoder.save('CheckAutoencoder.h5')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import sample
from PIL import Image  # Import Pillow for image dimension retrieval

combined_df = pd.read_csv('/root/ubc_ocean/alper/models/similar_tmas/similar_images_final_combined.csv')
random_image_paths = sample(combined_df['Similar_Image_Path'].unique().tolist(), 25)



plt.figure(figsize=(20, 20))

for i, img_path in enumerate(random_image_paths, 1):
    img = mpimg.imread(img_path)  # Make sure the img_path is accessible
    
    # Get image dimensions using Pillow
    with Image.open(img_path) as img_pillow:
        width, height = img_pillow.size
    
    plt.subplot(5, 5, i)  # Arrange images in a 5x5 grid
    plt.imshow(img)
    plt.text(5, 5, f'#{i}', color='black', fontsize=14, weight='bold')  # Adjusted position for number label
    plt.text(5, 180, f'{width} x {height}', color='black', fontsize=12)  # Adjusted position for image dimensions
    plt.axis('off')  # Hide axis

plt.tight_layout()
plt.show()



In [ ]:
from keras.models import load_model
import pandas as pd


autoencoder = load_model('CheckAutoencoderSimple.h5')

autoencoder.summary()


In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout
from keras.models import Model

# Build the encoder model from the existing autoencoder layers
encoder_input = autoencoder.input
#encoded_output = autoencoder.get_layer('conv2d_21').output  # This is the last layer of the encoder part
encoded_output = autoencoder.get_layer('conv2d_5').output


# Flatten the output of the encoder to feed into the dense layers
flattened = Flatten()(encoded_output)

# Add dense layers for classification
hidden = Dense(128, activation='relu')(flattened)
dropout = Dropout(0.5)(hidden)
output = Dense(5, activation='softmax')(dropout)  # num_classes is the number of your target classes

# Create a new model for classification
classifier_model = Model(inputs=encoder_input, outputs=output)

# Compile the model
classifier_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the summary of the new model
classifier_model.summary()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Load the data
tma_image_paths = pd.read_csv('/root/ubc_ocean/alper/models/paths/tma_image_paths.csv')
train_labels = pd.read_csv('/root/ubc/train.csv')

# Extract the image ID from the file path
tma_image_paths['image_id'] = tma_image_paths['img_path'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

# Convert the image_id in train_labels to string if it's not already
train_labels['image_id'] = train_labels['image_id'].astype(str)

# Merge the two DataFrames based on the image_id
merged_data = pd.merge(tma_image_paths, train_labels, on='image_id')

# Split your data into train, validation, and test sets ensuring at least one sample from each label in the test set
grouped = merged_data.groupby('label')
test_data = pd.concat([group.sample(1, random_state=42) for name, group in grouped])
train_val_data = merged_data.drop(test_data.index)
x_train, x_val = train_test_split(train_val_data, test_size=0.2, random_state=42)

# Convert these to lists of image paths for training the model
train_image_paths = x_train['img_path'].tolist()
train_labels = x_train['label'].values

val_image_paths = x_val['img_path'].tolist()
val_labels = x_val['label'].values

test_image_paths = test_data['img_path'].tolist()
test_labels = test_data['label'].values

train_labels

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout
from keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# Function to preprocess a single image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return np.squeeze(img_array)


# Extract features for each set
train_features = [preprocess_image(img_path) for img_path in train_image_paths]
val_features = [preprocess_image(img_path) for img_path in val_image_paths]
test_features = [preprocess_image(img_path) for img_path in test_image_paths]
train_features

In [ ]:
autoencoder.summary()

In [ ]:
# Convert the list to a NumPy array before normalization
train_features_array = np.array(train_features)
test_features_array = np.array(test_features)
# Normalize the features
train_features_normalized = train_features_array / 255.0
test_features_normalized = test_features_array / 255.0

# Checking the shape and type of the normalized features
shape_of_normalized_features = train_features_normalized.shape
type_of_normalized_features = train_features_normalized.dtype

shape_of_normalized_features_test = test_features_normalized.shape
type_of_normalized_features_test = test_features_normalized.dtype


# Output the shape and type
print("Shape of normalized train features:", shape_of_normalized_features)
print("Type of normalized train features:", type_of_normalized_features)


# Output the shape and type
print("Shape of normalized test features:", shape_of_normalized_features_test)
print("Type of normalized test features:", type_of_normalized_features_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming train_labels is an array of categorical labels
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(train_labels)
numeric_labels

numeric_labels_test = label_encoder.fit_transform(test_labels)
numeric_labels_test

from tensorflow.keras.utils import to_categorical

# Assuming there are 5 different classes
num_classes = 5
y_train = to_categorical(numeric_labels, num_classes=num_classes)
y_test = to_categorical(numeric_labels_test, num_classes=num_classes)
y_test

In [ ]:
# Training the model
history = classifier_model.fit(
    train_features_normalized, 
    y_train, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2  # Splitting the training data for validation
)

# Plotting training history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left') 
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Make predictions on the test set
test_predictions = classifier_model.predict(test_features_normalized)

# The `test_predictions` array will contain the predicted probabilities or labels,
# depending on the last layer of your model and the activation function used.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import torch

true_labels = np.argmax(y_test, axis=1)  # Only necessary if y_test is one-hot encoded
true_labels = np.array(true_labels)
predicted_labels = np.argmax(test_predictions, axis=1)


# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro', zero_division=1)
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

# Print the metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
# Assuming y_test contains the true labels
from sklearn.metrics import accuracy_score

true_labels = np.argmax(y_test, axis=1)  # Only necessary if y_test is one-hot encoded
accuracy = accuracy_score(true_labels, predicted_labels)

print("Test set accuracy:", accuracy)